[![](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-spatial/blob/main/docs/vitessce.ipynb) [![hub](https://img.shields.io/badge/View%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/hqtT4OTr5Tiq5zKv/URbaThYhljgXbZEzgj5x)

# Vitessce

This tutorial has been adopted from the data preparation examples in [the Vitessce documention](https://vitessce.github.io/vitessce-python).

It uses a [dataset](https://www.covid19cellatlas.org/index.healthy.html#habib17) from the COVID-19 Cell Atlas.

In [ ]:
# !pip install vitessce>=0.3.4
# !pip install 'lamindb[jupyter,aws,bionty]'
!lamin load laminlabs/lamin-dev  # <-- replace with your instance

In [ ]:
from urllib.request import urlretrieve
from pathlib import Path
from anndata import read_h5ad
import vitessce as vit
from vitessce import data_utils as vitdu
import lamindb as ln

# [optional] track the current notebook or script
ln.track("BZhZQ6uIbkWv0000")

## Save your dataset

Convert the dataset to `.zarr` format.

In [ ]:
# from https://github.com/vitessce/vitessce-python/blob/main/demos/habib-2017/src/convert_to_zarr.py
def convert_h5ad_to_zarr(input_path, output_path):
    adata = read_h5ad(input_path)
    adata = adata[:, adata.var["highly_variable"]].copy()
    leaf_list = vitdu.sort_var_axis(adata.X, adata.var.index.values)
    adata = adata[:, leaf_list].copy()
    adata.layers["X_uint8"] = vitdu.to_uint8(adata.X, norm_along="var")
    adata = vitdu.optimize_adata(
        adata, obs_cols=["CellType"], obsm_keys=["X_umap"], layer_keys=["X_uint8"]
    )
    adata.write_zarr(output_path)


h5ad_filepath = "./habib17.processed.h5ad"
if not Path(h5ad_filepath).exists():
    urlretrieve(
        "https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad", h5ad_filepath
    )
zarr_filepath = "./hhabib_2017_nature_methods.anndata.zarr"
convert_h5ad_to_zarr(h5ad_filepath, zarr_filepath)

Save a `.zarr` version of the dataset to lamindb.

In [ ]:
zarr_artifact = ln.Artifact(
    zarr_filepath,
    description="Habib et al., 2017 Nature Methods, optimized anndata zarr",
    type="dataset",
)
zarr_artifact.save()

## Save a VitessceConfig object

You can create a dashboard for one or several datasets by using Vitessce's component API.

In [ ]:
vc = vit.VitessceConfig(
    schema_version="1.0.15",
    description=zarr_artifact.description,
)
dataset = vc.add_dataset(name="Habib 2017").add_object(
    vit.AnnDataWrapper(
        adata_artifact=zarr_artifact,
        obs_feature_matrix_path="layers/X_uint8",
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
    )
)
obs_sets = vc.add_view(vit.Component.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(vit.Component.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(vit.Component.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(vit.Component.HEATMAP, dataset=dataset)
genes = vc.add_view(vit.Component.FEATURE_LIST, dataset=dataset)
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))

Save the `VitessceConfig` object.

In [ ]:
vc_artifact = ln.integrations.save_vitessce_config(
    vc, description="View Habib17 in Vitessce"
)

:::{note}

You can now see the Vitessce button show up on your dataset as in this [example dataset](https://lamin.ai/laminlabs/lamindata/artifact/HXJ4DDAw8012jVKwoxgd).

If your `VitessceConfig` object contains multiple datasets, the Vitessce button will appear next to a `Collection` that groups these artifacts.

:::

In [ ]:
vc_artifact.view_lineage()

In [ ]:
# [optional] finish run context and auto-save the notebook
# ln.finish()

:::{dropdown} Upload speed

Here is [a note](https://lamin.ai/laminlabs/lamindata/transform/WDjio16cQsdW5zKv) on folder upload speed and why `lamindb` does not use the `.export(to="s3")` functionality of Vitessce.

:::

In [ ]:
# clean up artifacts in CI run
zarr_artifact.delete(permanent=True)
vc_artifact.delete(permanent=True)